# Knowledge Representation on the Web -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [7]:
import sys
!{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef
from owlrl import DeductiveClosure, RDFS_Semantics

## Loading RDFS graphs

The file `rdfs.ttl` contains a basic Knowledge Graph in RDF containing statements with RDFS semantics.

In [5]:
g = Graph()
g.parse('rdfs.ttl', format='turtle')
print("Loaded graph has {} triples".format(len(g)))

Loaded graph has 9 triples


Inspecting the graph as-is; this is also called the "asserted graph"

In [8]:
#for s,p,o in sorted(g.triples((None,None,None))):
#    print(s,p,o)

# get the list classes 
uniqClasses = list()
for s,p, o in g.triples((None, RDF.type , None)):
    uniqClasses.append(o)
    
print("List of classes")
for cl in set(uniqClasses) : print (cl)
    
personProperty = list()
# let's look at properties of the class Person
for s,p,o in g.triples((None, None,URIRef("http://www.example.org/sample.rdfs#Person") )): #always remember to pass a string to URIRef ! 
    
    # now we look at which predicates does a Person has 
    for s1,p1,o1 in g.triples((s, None, None)):
        personProperty.append(p1)

print("\nProperties for the class 'Person'")
for pp in set(personProperty) : print (pp)
    

print("\nInstances in the dataset")
for s,p,o in g.triples((None, RDF.type, None )):
    print (s)


List of classes
http://www.example.org/sample.rdfs#Dog
http://www.example.org/sample.rdfs#Person

Properties for the class 'Person'
http://www.example.org/sample.rdfs#hasSon
http://www.w3.org/1999/02/22-rdf-syntax-ns#type

Instances in the dataset
http://www.example.org/sample.rdfs#Adam
http://www.example.org/sample.rdfs#Abel
http://www.example.org/sample.rdfs#Max


## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


In [4]:
DeductiveClosure(RDFS_Semantics).expand(g)
print("RDFS closure of the graph has {} triples".format(len(g)))

RDFS closure of the graph has 35 triples


In [9]:
for s,p,o in sorted(g.triples((None,None,None))):
    if o != RDFS.Resource:
        print(s,p,o)

http://www.example.org/sample.rdfs#Abel http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.example.org/sample.rdfs#Person
http://www.example.org/sample.rdfs#Adam http://www.example.org/sample.rdfs#hasSon http://www.example.org/sample.rdfs#Abel
http://www.example.org/sample.rdfs#Adam http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.example.org/sample.rdfs#Person
http://www.example.org/sample.rdfs#Dog http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.example.org/sample.rdfs#Animal
http://www.example.org/sample.rdfs#Max http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.example.org/sample.rdfs#Dog
http://www.example.org/sample.rdfs#Person http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.example.org/sample.rdfs#Animal
http://www.example.org/sample.rdfs#hasChild http://www.w3.org/2000/01/rdf-schema#domain http://www.example.org/sample.rdfs#Animal
http://www.example.org/sample.rdfs#hasChild http://www.w3.org/2000/01/rdf-schema#range http://www.exa

## Exercise -- the explicit (asserted) and the implicit (derived)

Write here code to generate a graph that contains **RDFS derived triples only** from the Knowledge Graph at `rdfs.ttl`, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)

In [11]:
## Your answer here (correct answer in rdfs-ex.py -- don't look until you've written yours!)
u = Graph()
v = Graph()
d = Graph()

u.parse('rdfs.ttl', format='turtle')
v.parse('rdfs.ttl', format='turtle')

DeductiveClosure(RDFS_Semantics).expand(v)
d = v - u

print("RDFS inference generated additional {} triples".format(len(d)))

for s,p,o in sorted(d.triples((None,None,None))):
    print(s,p,o)

RDFS inference generated additional 26 triples
http://www.example.org/sample.rdfs#Abel http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.example.org/sample.rdfs#Animal
http://www.example.org/sample.rdfs#Abel http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.example.org/sample.rdfs#Adam http://www.example.org/sample.rdfs#hasChild http://www.example.org/sample.rdfs#Abel
http://www.example.org/sample.rdfs#Adam http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.example.org/sample.rdfs#Animal
http://www.example.org/sample.rdfs#Adam http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.example.org/sample.rdfs#Animal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.example.org/sample.rdfs#Dog http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.example.org/sample.rd

## Exercise -- explain derivations

Based on the RDFS semantics, explain the rule that was applied to generate each derivation of the previous exercise

## RDF Entailment

Suppose you have the following :

ex:book1 rdf:type ex:Publication . 
ex:book2 rdf:type ex:Article .
            
So a Sparql query like SELECT ?s { ?s rdf:type ex:Publication } will return only ex:book1

If you add the fact (or a graph in your data set with the fact) that states :

ex:Article rdfs:subClassOf ex:Publication
If your sparql engine processes entailments, it should deduce that a ex:Article is also a ex:Publication

so SELECT ?s { ?s rdf:type ex:Publication } would return both ex:book1 AND ex:book2